In [ ]:
from itertools import chain
import copy
import json
import nltk
import glob
import pandas as pd
import sklearn
import spacy
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report
# from sklearn.cross_validation import cross_val_score
# from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
model_dir = '/Users/talhindi/miniconda3/lib/python3.7/site-packages/en_core_web_sm/en_core_web_sm-2.1.0'

In [ ]:
# reading essays
train_test_split = pd.read_csv('../data/SG2017/train-test-split.csv', sep=';')

essays_txt_prg_list = []
for file in sorted(glob.glob("../data/SG2017/*.txt")):
    essay = open(file).readlines()
    essays_txt_prg_list.append(essay)

essay_txt_str = []
for essay in essays_txt_prg_list:
    essay_txt_str.append(''.join(essay))

In [ ]:
# tokenization
nlp = spacy.load(model_dir)

essay_spacy = []
for essay in essay_txt_str:
    essay_spacy.append(nlp(essay))

In [ ]:
# getting sent_splits for train and test
token_id = 0
sent_token_ids_train, sent_token_ids_test = [], []

for i, (doc, group) in enumerate(zip(essay_spacy, train_test_split.SET)):
    
    if group == "TRAIN":
        for sent in doc.sents:
            sent_tokens_ids = []
            for token in sent:
                sent_tokens_ids.append(token_id)
                token_id +=1
            sent_token_ids_train.append(sent_tokens_ids)
    
    else:
        for sent in doc.sents:
            sent_tokens_ids = []
            for token in sent:
                sent_tokens_ids.append(token_id)
                token_id +=1
            sent_token_ids_test.append(sent_tokens_ids)
    

## Functions

In [5]:
def read_features_labels(n_examples, sent_token_ids, directory, feature_files, suffix):
    features, labels, feature_sets = [], [], []
    
    for file in feature_files:
        feature_sets.append(open(directory + file + suffix, 'r').readlines())
    
    for fset in feature_sets:
        assert len(fset) == n_examples
    
    sent_id, token_id = 0, 0
    sent, sent_labels = [], []
    for i in range(n_examples):
        
        token_features = {}
        for fset in feature_sets:
            jsonline = json.loads(fset[i])
            for key in jsonline['x'].keys():
                if type(jsonline['x'][key]) is int:
                    token_features[key] = float(jsonline['x'][key])
                else:
                    token_features[key] = jsonline['x'][key]
        
#         print(sent_id, token_id, sent_token_ids[sent_id][token_id], jsonline['id'])
        assert sent_token_ids[sent_id][token_id] == jsonline['id']
        
        sent.append(copy.deepcopy(token_features))
        sent_labels.append(str(jsonline['y']))
        del token_features
        
        if token_id == len(sent_token_ids[sent_id])-1:
#             print(jsonline['id'], len(sent), len(sent_labels), len(sent_token_ids[sent_id]))
            assert len(sent) == len(sent_labels) == len(sent_token_ids[sent_id])
            
            features.append(sent)
            labels.append(sent_labels)
            
            sent, sent_labels = [], []
            sent_id += 1; token_id = 0
        else:
            token_id += 1
    
    return features, labels

In [14]:
features = ["structural_position", "structural_position_sent", "structural_punc",
            "syntactic_LCA_bin", "syntactic_POS", "syntactic_LCA_type", "lexsyn_1hop"]

suffix='.jsonlines'
train_test_sent_splits = [sent_token_ids_train, sent_token_ids_test]
train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                        '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/']

n_train, n_test = 119752, 29815
n_train_test = [n_train, n_test]

# X_train, y_train = read_features_labels(n_train_test[0], train_test_sent_splits[0],
#                                             train_test_directories[0], features, suffix)
# X_test, y_test = read_features_labels(n_train_test[1], train_test_sent_splits[1],
#                                             train_test_directories[1], features, suffix)

In [13]:
import pickle
pickle.dump(X_train, open('../features/train_all.p','wb'))
pickle.dump(X_test, open('../features/test_all.p','wb'))
pickle.dump(y_train, open('../features/train_labels.p','wb'))
pickle.dump(y_test, open('../features/test_labels.p','wb'))

In [15]:
features = ["lexsyn_1hop"]
X_train, y_train = read_features_labels(n_train_test[0], train_test_sent_splits[0],
                                            train_test_directories[0], features, suffix)
X_test, y_test = read_features_labels(n_train_test[1], train_test_sent_splits[1],
                                            train_test_directories[1], features, suffix)

In [16]:
pickle.dump(X_train, open('../features/train_lexsyn.p','wb'))
pickle.dump(X_test, open('../features/test_lexsyn.p','wb'))

# CRF experiments

In [6]:
n_train, n_test = 119752, 29815

def run_crf_expriments(features, suffix='.jsonlines',
                   train_test_sent_splits = [sent_token_ids_train, sent_token_ids_test],
                   train_test_directories = ['../features/SG2017_train/', '../features/SG2017_test/'],
                   n_train_test = [n_train, n_test]):
    
    X_train, y_train = read_features_labels(n_train_test[0], train_test_sent_splits[0],
                                            train_test_directories[0], features, suffix)
    X_test, y_test = read_features_labels(n_train_test[1], train_test_sent_splits[1],
                                            train_test_directories[1], features, suffix)
    
    print('train dir: ', train_test_directories[0])
    print('test dir: ', train_test_directories[1])
    print('\nReading the following features: ')
    print(features)
    
    print('\nBuilding  CRF model')
    crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
    )
    print(crf)
    
    print('\nTraining Model...')
    crf.fit(X_train, y_train)
    
    print('\nPredictions on the test set')
    y_pred = crf.predict(X_test)
    print('Macro F1: ', metrics.flat_f1_score(y_test, y_pred, average='macro', labels=['0.0','1.0','2.0']))
    
    print('\nclassification report:')
    y_test_flat = [y for y_seq in y_test for y in y_seq]
    y_pred_flat = [y for y_seq in y_pred for y in y_seq]
    print(classification_report(y_test_flat, y_pred_flat, digits=3))
    
    
    print(metrics.flat_classification_report(y_test, y_pred, labels=['0.0','1.0','2.0'], digits=3))

## Claim-only Experiments: structural

In [11]:
run_crf_expriments(["structural_position"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['structural_position']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.30270967814272076

classification report:
              precision    recall  f1-score   support

         0.0      0.789     1.000     0.882     23470
         1.

In [12]:
run_crf_expriments(["structural_position_sent"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['structural_position_sent']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.29777697095688044

classification report:


/Users/talhindi/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0      0.788     0.995     0.879     23470
         1.0      0.000     0.000     0.000       457
         2.0      0.254     0.007     0.014      5888

    accuracy                          0.785     29815
   macro avg      0.347     0.334     0.298     29815
weighted avg      0.670     0.785     0.695     29815

              precision    recall  f1-score   support

         0.0      0.788     0.995     0.879     23470
         1.0      0.000     0.000     0.000       457
         2.0      0.254     0.007     0.014      5888

    accuracy                          0.785     29815
   macro avg      0.347     0.334     0.298     29815
weighted avg      0.670     0.785     0.695     29815



In [13]:
run_crf_expriments(["structural_punc"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['structural_punc']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.2937107617087332

classification report:
              precision    recall  f1-score   support

         0.0      0.785     0.984     0.873     23470
         1.0    

In [10]:
run_crf_expriments(["structural_position", "structural_position_sent", "structural_punc"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['structural_position', 'structural_position_sent', 'structural_punc']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.4134290053439565

classification report:
              precision    recall  f1-score   support

         0.0      0

## Claim-only Experiments: Syntactic

In [14]:
run_crf_expriments(["syntactic_LCA_bin"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['syntactic_LCA_bin']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.30390698350710527

classification report:
              precision    recall  f1-score   support

         0.0      0.788     0.989     0.877     23470
         1.0 

In [15]:
run_crf_expriments(["syntactic_POS"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['syntactic_POS']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.31482970227275936

classification report:
              precision    recall  f1-score   support

         0.0      0.788     0.966     0.868     23470
         1.0     

In [16]:
run_crf_expriments(["syntactic_LCA_type"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['syntactic_LCA_type']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.34211133365596685

classification report:
              precision    recall  f1-score   support

         0.0      0.788     0.955     0.864     23470
         1.0

In [17]:
run_crf_expriments(["syntactic_LCA_bin", "syntactic_POS", "syntactic_LCA_type"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['syntactic_LCA_bin', 'syntactic_POS', 'syntactic_LCA_type']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.3799836482576798

classification report:
              precision    recall  f1-score   support

         0.0      0.795     0

## Claim-only Experiments: LexSyn

In [7]:
run_crf_expriments(["lexsyn_1hop"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['lexsyn_1hop']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...


/Users/talhindi/miniconda3/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)



Predictions on the test set
Macro F1:  0.5375267378485489

classification report:
              precision    recall  f1-score   support

         0.0      0.833     0.899     0.865     23470
         1.0      0.457     0.293     0.357       457
         2.0      0.468     0.335     0.391      5888

    accuracy                          0.778     29815
   macro avg      0.586     0.509     0.538     29815
weighted avg      0.755     0.778     0.763     29815

              precision    recall  f1-score   support

         0.0      0.833     0.899     0.865     23470
         1.0      0.457     0.293     0.357       457
         2.0      0.468     0.335     0.391      5888

    accuracy                          0.778     29815
   macro avg      0.586     0.509     0.538     29815
weighted avg      0.755     0.778     0.763     29815



In [8]:
run_crf_expriments(["lexsyn_2hops"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['lexsyn_2hops']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.49812287113819925

classification report:
              precision    recall  f1-score   support

         0.0      0.820     0.907     0.862     23470
         1.0      

## Claim-only Experiment: All features

In [9]:
run_crf_expriments(["structural_position", "structural_position_sent", "structural_punc",
                    "syntactic_LCA_bin", "syntactic_POS", "syntactic_LCA_type", "lexsyn_1hop"],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/claim_features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/claim_features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/claim_features/test/

Reading the following features: 
['structural_position', 'structural_position_sent', 'structural_punc', 'syntactic_LCA_bin', 'syntactic_POS', 'syntactic_LCA_type', 'lexsyn_1hop']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.5732142518124882

classification report:

# Arg Experiments

In [10]:
run_crf_expriments(['probability'],
                   train_test_directories=['/Users/talhindi/Documents/claim_detection_wm/features/train/',
                                          '/Users/talhindi/Documents/claim_detection_wm/features/test/'])

train dir:  /Users/talhindi/Documents/claim_detection_wm/features/train/
test dir:  /Users/talhindi/Documents/claim_detection_wm/features/test/

Reading the following features: 
['probability']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...


/Users/talhindi/miniconda3/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)



Predictions on the test set
Macro F1:  0.257375451313098

classification report:
              precision    recall  f1-score   support

         0.0      0.000     0.000     0.000      9801
         1.0      0.000     0.000     0.000      1266
         2.0      0.629     1.000     0.772     18748

    accuracy                          0.629     29815
   macro avg      0.210     0.333     0.257     29815
weighted avg      0.395     0.629     0.486     29815



/Users/talhindi/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# structural features
run_crf_expriments(['position', 'punc', 'position_sent'])

train dir:  ../features/SG2017_train/
test dir:  ../features/SG2017_test/

Reading the following features: 
['position', 'punc', 'position_sent']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.7379007614294025

classification report:
              precision    recall  f1-score   support

         0.0      0.813     0.642     0.718      9801
         1.0      0.642     0.598     0.619      1266
         2.0      0.832   

In [17]:
# syntactic features
run_crf_expriments(['POS', 'LCA_bin', 'LCA_type'])

train dir:  ../features/SG2017_train/
test dir:  ../features/SG2017_test/

Reading the following features: 
['POS', 'LCA_bin', 'LCA_type']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.6747796409086603

classification report:
              precision    recall  f1-score   support

         0.0      0.630     0.623     0.627      9801
         1.0      0.638     0.535     0.582      1266
         2.0      0.809     0.823

In [18]:
# structural + syntactic features
run_crf_expriments(['position', 'punc', 'position_sent', 'POS', 'LCA_bin', 'LCA_type'])

train dir:  ../features/SG2017_train/
test dir:  ../features/SG2017_test/

Reading the following features: 
['position', 'punc', 'position_sent', 'POS', 'LCA_bin', 'LCA_type']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.7601211693498886

classification report:
              precision    recall  f1-score   support

         0.0      0.743     0.741     0.742      9801
         1.0      0.707     0.635     0.669      1

In [23]:
# lexsyn... mistake.. only dep relation, no lexical info
run_crf_expriments(['LexSyn'])

train dir:  ../features/SG2017_train/
test dir:  ../features/SG2017_test/

Reading the following features: 
['LexSyn']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.7063305796204723

classification report:
              precision    recall  f1-score   support

         0.0      0.751     0.588     0.659      9801
         1.0      0.622     0.588     0.605      1266
         2.0      0.810     0.905     0.855     18748

In [24]:
# lexsyn 1hop, same as above, sanity check... hmmm, slightly better!
run_crf_expriments(['LexSyn_1hop'])

train dir:  ../features/SG2017_train/
test dir:  ../features/SG2017_test/

Reading the following features: 
['LexSyn_1hop']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.7168117709637286

classification report:
              precision    recall  f1-score   support

         0.0      0.760     0.587     0.662      9801
         1.0      0.644     0.619     0.631      1266
         2.0      0.810     0.909     0.857     

In [26]:
# lexsyn 1hop, with word
run_crf_expriments(['LexSyn_1hop'])

train dir:  ../features/SG2017_train/
test dir:  ../features/SG2017_test/

Reading the following features: 
['LexSyn_1hop']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.7971826180542054

classification report:
              precision    recall  f1-score   support

         0.0      0.795     0.698     0.743      9801
         1.0      0.763     0.768     0.765      1266
         2.0      0.857     0.911     0.883     

In [27]:
# lexsyn 2hops, with word

run_crf_expriments(['LexSyn_2hops'])

train dir:  ../features/SG2017_train/
test dir:  ../features/SG2017_test/

Reading the following features: 
['LexSyn_2hops']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.7886071128590383

classification report:
              precision    recall  f1-score   support

         0.0      0.772     0.692     0.730      9801
         1.0      0.759     0.762     0.761      1266
         2.0      0.853     0.899     0.875    

In [28]:
run_crf_expriments(['position', 'punc', 'position_sent', 'POS', 'LCA_bin', 'LCA_type','LexSyn_1hop'])

train dir:  ../features/SG2017_train/
test dir:  ../features/SG2017_test/

Reading the following features: 
['position', 'punc', 'position_sent', 'POS', 'LCA_bin', 'LCA_type', 'LexSyn_1hop']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.8183587608396903

classification report:
              precision    recall  f1-score   support

         0.0      0.777     0.790     0.783      9801
         1.0      0.806     0.758  

In [29]:
run_crf_expriments(['position', 'punc', 'position_sent','LexSyn_1hop'])

train dir:  ../features/SG2017_train/
test dir:  ../features/SG2017_test/

Reading the following features: 
['position', 'punc', 'position_sent', 'LexSyn_1hop']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.8186417498379485

classification report:
              precision    recall  f1-score   support

         0.0      0.788     0.773     0.780      9801
         1.0      0.810     0.759     0.784      1266
         2.

In [30]:
run_crf_expriments(['POS', 'LCA_bin', 'LCA_type','LexSyn_1hop'])

train dir:  ../features/SG2017_train/
test dir:  ../features/SG2017_test/

Reading the following features: 
['POS', 'LCA_bin', 'LCA_type', 'LexSyn_1hop']

Building  CRF model
CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Training Model...

Predictions on the test set
Macro F1:  0.7989099608292016

classification report:
              precision    recall  f1-score   support

         0.0      0.796     0.698     0.744      9801
         1.0      0.769     0.769     0.769      1266
         2.0      